In [121]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [122]:
df = pd.read_csv('data/AAPL.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099319,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094137,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087228,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089387,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091978,73449600
...,...,...,...,...,...,...,...
10829,2023-11-27,189.919998,190.669998,188.899994,189.789993,189.789993,40552600
10830,2023-11-28,189.779999,191.080002,189.399994,190.399994,190.399994,38415400
10831,2023-11-29,190.899994,192.089996,188.970001,189.369995,189.369995,43014200
10832,2023-11-30,189.839996,190.320007,188.190002,189.949997,189.949997,48794400


In [123]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,10834.000000,10834.000000,10834.000000,10834.000000,10834.000000,1.083400e+04
mean,19.752257,19.970329,19.543086,19.765683,19.013750,3.219534e+08
std,41.335900,41.800261,40.905323,41.373839,40.966940,3.364729e+08
min,0.049665,0.049665,0.049107,0.049107,0.038000,0.000000e+00
25%,0.294643,0.301339,0.286997,0.294643,0.241206,1.164632e+08
50%,0.508929,0.515625,0.500000,0.507813,0.417317,2.094400e+08
75%,18.760892,18.960625,18.635625,18.791787,16.364081,4.018196e+08
max,196.240005,198.229996,195.279999,196.449997,195.926956,7.421641e+09


In [124]:
df = df[['Date', 'Close']]
df

,Date,Close
0,1980-12-12,0.128348
1,1980-12-15,0.121652
2,1980-12-16,0.112723
3,1980-12-17,0.115513
4,1980-12-18,0.118862
...,...,...
10829,2023-11-27,189.789993
10830,2023-11-28,190.399994
10831,2023-11-29,189.369995
10832,2023-11-30,189.949997


In [125]:
#Function to convert string dates into proper datetime objects
import datetime

def str_to_datetime(date):
    split = date.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return datetime.datetime(year=year, month=month, day=day)


In [126]:
#Transform date column to the datetime dtype
df['Date'] = df['Date'].apply(str_to_datetime)
df['Date']

/var/folders/k1/bv0_2yxx3m54j48_94tn2g140000gn/T/ipykernel_3315/1060664854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df['Date'].apply(str_to_datetime)


0       1980-12-12
1       1980-12-15
2       1980-12-16
3       1980-12-17
4       1980-12-18
           ...    
10829   2023-11-27
10830   2023-11-28
10831   2023-11-29
10832   2023-11-30
10833   2023-12-01
Name: Date, Length: 10834, dtype: datetime64[ns]

In [127]:
df.index = df.pop('Date')
df

,Close
Date,
1980-12-12,0.128348
1980-12-15,0.121652
1980-12-16,0.112723
1980-12-17,0.115513
1980-12-18,0.118862
...,...
2023-11-27,189.789993
2023-11-28,190.399994
2023-11-29,189.369995


In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.index, df['Close'])